In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.6 MB/s eta 0:00:00

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [5]:
DATA_ROOT = "/content/drive/My Drive/dataset/"
df = pd.read_csv(DATA_ROOT+'toxic_classification_preprocessed.csv',names = ['target','comment_text','preprocessed_text'], skiprows = 1)
#df = df.dropna(axis = 0, how = 'any')
df_0 = df[df['target'] == 0].iloc[:20000, :]
df_1 = df[df['target'] == 1].iloc[:20000, :]
df_small = pd.concat([df_0, df_1])


In [6]:
train, test = train_test_split(df_small, test_size = 0.2, random_state = 0)

In [7]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [8]:
train.head()

,target,comment_text,preprocessed_text
0,1,One anecdotal story does not make a fact. FACT...,one anecdot stori make fact fact plan parentho...
1,0,Good for these republicans! The democrats are...,good republican democrat tri forc major legisl...
2,0,"A ""conservative reading""? Yup, heard enough of...",conserv read yup heard enough fail bad
3,0,I'm ok with our only Congressman going to a pa...,ok congressman go parti rich folk celebr alask...
4,1,Rump voters don't seem to think. They do watch...,rump voter seem think watch lot tv


In [9]:

# train = df.sample(frac=0.1, random_state=42).reset_index(drop=True)
# test = df.head(10000)
class ToxicDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.texts = self.data.comment_text
        self.labels = self.data.target
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = self.texts[idx]
        sentiment = self.labels[idx]
        return {'text': review, 'sentiment': sentiment}


train_dataset = ToxicDataset(train)
test_dataset = ToxicDataset(test)

# class ToxicDataset(Dataset):
#     def __init__(self, csv_file):
#         DATA_ROOT = "/content/drive/My Drive/dataset/"
#         self.data = pd.read_csv(DATA_ROOT+csv_file,names = ['target','comment_text','preprocessed_text'], skiprows = 1)
#         self.data = self.data.dropna(axis = 0, how = 'any')
#         self.data = self.data.sample(frac=0.001, random_state=42).reset_index(drop=True)
#         self.texts = self.data.preprocessed_text
#         self.labels = self.data.target
#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         review = self.texts[idx]
#         sentiment = self.labels[idx]
#         return {'text': review, 'sentiment': sentiment}


In [10]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [11]:
print(f"Train Dataset Length: {len(train_dataset)}")
print(f"Test Dataset Length: {len(test_dataset)}")

Train Dataset Length: 32000
Test Dataset Length: 8000


In [12]:
loss_fn = torch.nn.CrossEntropyLoss()

In [13]:
num_epoch = 0
num_batch = 0
for epoch in range(3):  # Adjust the number of training epochs as needed
    model.train()
    num_batch = 0
    for batch in train_loader:
        
        try:
          inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
        except:
          print(batch['text'])
          print("error length:   ", len(batch['text']))

        #inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
    
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = batch['sentiment'].to(device)

        model.zero_grad()

        optimizer.zero_grad()
        outputs = model(**inputs)
        #print(labels)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()

        
        optimizer.step()
        if (num_batch % 100) == 0:
          print("end: ", num_batch)
          print("Loss: ", loss)
        num_batch += 1
    print("End of epoch")


end:  0
Loss:  tensor(0.9037, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  100
Loss:  tensor(0.6071, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  200
Loss:  tensor(0.5104, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  300
Loss:  tensor(0.1624, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  400
Loss:  tensor(0.2143, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  500
Loss:  tensor(0.3044, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  600
Loss:  tensor(0.3873, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  700
Loss:  tensor(0.1545, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  800
Loss:  tensor(0.2441, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  900
Loss:  tensor(0.1703, device='cuda:0', grad_fn=<NllLossBackward0>)
End of epoch
end:  0
Loss:  tensor(0.1516, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  100
Loss:  tensor(0.0850, device='cuda:0', grad_fn=<NllLossBackward0>)
end:  200
Loss:  tensor(0.1485, device='cuda:0', grad_fn=<NllLossBa

In [14]:
# Step 6: Evaluation
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = batch['sentiment'].to(device)

        outputs = model(**inputs)
        _, predicted = torch.max(outputs.logits, 1)

        true_labels.extend(labels.tolist())
        predicted_labels.extend(predicted.tolist())

classification_rep = classification_report(true_labels, predicted_labels)
print(f'Classification Report:\n{classification_rep}')


Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      4004
           1       0.92      0.91      0.92      3996

    accuracy                           0.92      8000
   macro avg       0.92      0.92      0.92      8000
weighted avg       0.92      0.92      0.92      8000



In [15]:
classification_rep = classification_report(true_labels, predicted_labels, digits = 3)
print(f'Classification Report:\n{classification_rep}')

Classification Report:
              precision    recall  f1-score   support

           0      0.913     0.925     0.919      4004
           1      0.923     0.912     0.918      3996

    accuracy                          0.918      8000
   macro avg      0.918     0.918     0.918      8000
weighted avg      0.918     0.918     0.918      8000



In [9]:
def print_wrong_predictions_1to0(test_data, true_labels, predicted_labels):
    count = 0
    for index, row in test_data.iterrows():
        data = row['comment_text']  # Assuming 'text' is the column name containing the text data
        true_label = true_labels[index]
        predicted_label = predicted_labels[index]
        
        if true_label == 1 and predicted_label == 0:
            count += 1
            print("Index: ", index)
            print("Text:", data)
            print("True Label:", true_label)
            print("Predicted Label:", predicted_label)
            print("--------------------")
    print("Total count 1 to 0: ", count)
    

def print_wrong_predictions_0to1(test_data, true_labels, predicted_labels):
    count = 0
    for index, row in test_data.iterrows():
        data = row['comment_text']  # Assuming 'text' is the column name containing the text data
        true_label = true_labels[index]
        predicted_label = predicted_labels[index]
        
        if true_label == 0 and predicted_label == 1:
            count += 1
            print("Index: ", index)
            print("Text:", data)
            print("True Label:", true_label)
            print("Predicted Label:", predicted_label)
            print("--------------------")
    print("Total count 0 to 1: ", count)

def print_wrong_predictions(test_data, true_labels, predicted_labels):
    for index, row in test_data.iterrows():
        data = row['comment_text']  # Assuming 'text' is the column name containing the text data
        true_label = true_labels[index]
        predicted_label = predicted_labels[index]
        
        if true_label != predicted_label:
            print("Text:", data)
            print("True Label:", true_label)
            print("Predicted Label:", predicted_label)
            print("--------------------")



In [17]:
dict = {'true_labels': true_labels, 'predicted_labels': predicted_labels}  
df = pd.DataFrame(dict) 
df.to_csv('prediction.csv',index=False)

In [11]:
df1 = pd.read_csv('prediction.csv')

In [12]:
trueLabels = df1['true_labels'].tolist()
predictedLabels = df1['predicted_labels'].tolist()

In [13]:
print_wrong_predictions_1to0(test, trueLabels, predictedLabels)

Index:  34
Text: George Carlin had a great bit on prostitution which, as usual for Carlin, nails it. 
"Selling is legal. F#@%ing is legal. Why isn't selling f#@%ing legal? Why is it illegal to sell something that is perfectly legal to give away?"
True Label: 1
Predicted Label: 0
--------------------
Index:  55
Text: Since they reflexively vote republican no matter the names of the candidates, the majority of Alaskans have the government they deserve; the rest of us...not so much. 

Too many Alaska republicans think there's absolutely nothing wrong with oil industry employees in the legislature making decisions on oil taxes and credits. Too many Alaska republicans think its just fine for conservative legislators to use the power of government to force their religious beliefs on the rest of us. 

And too many Alaska republicans who are now complaining about state government have been voting for the same people over and over again for many years. According to these republicans, its never 

In [14]:
print_wrong_predictions_0to1(test, trueLabels, predictedLabels)

Index:  15
Text: What do you think?  You're the trump fanboi.
True Label: 0
Predicted Label: 1
--------------------
Index:  21
Text: I'd be more supportive of Miss Loretta if she'd said "why are there more Blacks in the system in the first place?" And "the frequency of the use of force in our jails is unacceptable no matter what color offenders' skin is". 

Instead it sounds like the Oscars; everyone talkin bout boycotting due to the lack of "diversity" and honoring more "people of color", when really they're advocating only for Blacks only, not Hispanics/Latinos, or Asians, or Natives, who are the more discriminated against by far.
True Label: 0
Predicted Label: 1
--------------------
Index:  38
Text: Well muckamuck, that was a very self defeating thing to say. Just saying....
True Label: 0
Predicted Label: 1
--------------------
Index:  42
Text: I hope the dog got a nice treat to get the taste of criminal out of his mouth.
True Label: 0
Predicted Label: 1
--------------------
Index: 

In [16]:
test.iloc[883]

target                              0
comment_text         Dang Christians.
preprocessed_text      dang christian
Name: 883, dtype: object